In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow as tf
from sklearn import decomposition

In [21]:
labels =  ['call_me', 'fingers_crossed', 'okay', 'paper', 'peace', 'rock', 'rock_on', 'scissor', 'thumbs', 'up']

In [29]:
images = []
train_labels = []
for label_name in labels:
    folder_path = 'datasets/HandGesture/images/' + label_name
    for filename in os.listdir(folder_path):
        img = Image.open(os.path.join(folder_path, filename))
        if img.mode != 'L':
            img = img.convert('L')
        img_arr = np.array(img)
        images.append(img_arr)
        train_labels.append(label_name)
train_images = np.array(images)

In [31]:
X = train_images.reshape(train_images.shape[0], -1)

In [33]:
X.shape

(5243, 46800)

In [34]:
train_images.shape

(5243, 195, 240)

# Feature Extraction using PCA

In [38]:
standardized_scalar = StandardScaler()
standardized_data = standardized_scalar.fit_transform(X)
standardized_data.shape

(5243, 46800)

In [39]:
cov_matrix = np.matmul(standardized_data.T, standardized_data)
cov_matrix.shape

(46800, 46800)